# Objetivo

* Nessa parte do projeto desenvolvido será feito o tuning de 4 modelos de machine learning. No caso, os modelos utilizados foram a Floresta aleatoria, Xgboost e MLP. Também será feito um Staking com o primeiro nível. No seu segundo nível será testado os modelos de regressão lógistica, Xgboost e MLP.

# Instalando a bibliotecas

In [ ]:
!pip install git+git://github.com/Hvass-Labs/scikit-optimize.git@dd7433da068b5a2509ef4ea4e5195458393e6555`

/bin/bash: -c: line 0: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 6.2 MB/s 


In [ ]:
!pip install category_encoders

     |████████████████████████████████| 86 kB 3.6 MB/s 


#Funções

In [ ]:
def Tuning_RF(params):
  max_depth = params[0]
  n_estimators = params[1]  
  max_features = params[2]

  print('max_depth:',max_depth)
  print('n_estimators:',n_estimators)
  print('max_features:',max_features)

  model = RandomForestClassifier(n_estimators = n_estimators,max_features = max_features,max_depth=max_depth,random_state = 10) # modelo padrão
  f1   = resultados(X_train,y_train,model,0.36)
  return -f1


def Tuning_XGBR(params):
  lr = params[0]
  max_depth = params[1]
  n_estimators = params[2]
  colsample_bytree = params[3]



  print('lr:',params[0])
  print('max_depth:',params[1])
  print('n_estimators:',params[2])
  print('colsample_bytree:',params[3])
  
  model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,random_state=40)
  f1   = resultados(X_train,y_train,model,0.36)
  return -f1

def Tuning_MLP(params):
  activation = params[0]
  hidden_layer_sizes = params[1]
  solver = 'adam'
  alpha = params[2]
  print('activation:',params[0])
  print('hidden_layer_sizes:',params[1])
  print('solver:','adam')
  print('alpha:',params[2])
  
  clf   = MLPClassifier(hidden_layer_sizes =[hidden_layer_sizes],activation=activation,solver=solver,alpha=alpha,max_iter=500,random_state=40)
  model = make_pipeline(MinMaxScaler(), clf)

  f1   = resultados(X_train,y_train,model,0.36)
  return -f1


def Tuning_XGBR1(params):
  lr = params[0]
  max_depth = params[1]
  n_estimators = params[2]
  colsample_bytree = params[3]
  lamb             = params[4]
  alpha            = params[5]


  print('lr:',params[0])
  print('max_depth:',params[1])
  print('n_estimators:',params[2])
  print('colsample_bytree:',params[3])
  print('reg_lambda:',params[4])
  print('reg_alpha:',params[5])

  model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)
  f1   = resultados(X_train,y_train,model)
  return -f1



def resultados(X_train,y_train,model,C):
  j= 0
  acerto_v = np.zeros(10)
  for linhas_treino, linhas_valid in kf.split(X_train):
    X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
    y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

    model = model.fit(X_t, y_t)
    prf = model.predict_proba(X_v)[:,1]
    prf_ = (prf > C).astype(int)
    acerto_v[j] = f1_score(y_v, prf_)
    j = j + 1
  return np.mean(acerto_v)

def resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,C):
  X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]
  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > C).astype(int)
  acerto_v = f1_score(y_v, prf_)
  print('f1_score',np.format_float_positional(acerto_v,5))
  return acerto_v


def resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model):
  X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]
  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  return prf

def tratamento(df, flag):

  df['func_idade_log'] = np.log(df['func_idade'])

  df['contrato_salario_log'] = np.log(df['contrato_salario'])

  


  if flag == 1:
    df['turnover_apos_1_ano'] = df['turnover_apos_1_ano'].map({'NAO': 0, 'SIM': 1})
  
  df['func_deficiencia'] = df['func_deficiencia'].map({'NAO': 0, 'SIM': 1})
  df['func_sexo'] = df['func_sexo'].map({'FEMININO': 0, 'MASCULINO': 1})


  df = encoder_var(df,1)

  

  col2 = ['func_idade' ,'contrato_salario_log']
  df = df.drop(columns = col2)

  df['contrato_horastrabalho'].loc[df['contrato_horastrabalho'] == 0] =df['contrato_horastrabalho'].loc[df['contrato_horastrabalho'] > 0].mean()

  df['contrato_horastrabalho']= pd.cut(df["contrato_horastrabalho"], bins= 12)
  
  #df['func_racacor'][df['func_racacor'] =='SEM INFO'] = 'INDIGENA'

  print(np.unique(df['contrato_horastrabalho']))

  #df['contrato_salario_sqrt']= df['contrato_salario']**(1/3)

  df = encoder_var(df,0)

  #df = df.drop(columns = 'func_deficiencia')

  return df


def encoder_var(df,flag):
  if flag == 1:
    ind = df.columns[df.dtypes == 'object']
  if flag == 0:
     ind = df.columns[df.dtypes == 'category']

  for i in range(len(ind)):
    enc = LabelEncoder()
    inteiros = enc.fit_transform(df[ind[i]])
    df[ind[i]] = inteiros

  return df


# Bibliotecas

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from skopt import gp_minimize
from sklearn.neighbors import KNeighborsClassifier
from category_encoders import CountEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Importando os dados

In [ ]:
df = pd.read_csv('/content/drive/My Drive/competição flai nova/treino.csv',encoding = "UTF-8", sep = ";",decimal = ',',dayfirst = True)

In [ ]:
#np.log(df['contrato_salario']

In [ ]:
df = tratamento(df,1)

[Interval(0.957, 4.583, closed='right')
 Interval(4.583, 8.167, closed='right')
 Interval(8.167, 11.75, closed='right')
 Interval(11.75, 15.333, closed='right')
 Interval(15.333, 18.917, closed='right')
 Interval(18.917, 22.5, closed='right')
 Interval(22.5, 26.083, closed='right')
 Interval(26.083, 29.667, closed='right')
 Interval(29.667, 33.25, closed='right')
 Interval(33.25, 36.833, closed='right')
 Interval(36.833, 40.417, closed='right')
 Interval(40.417, 44.0, closed='right')]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Separando os dados

In [ ]:
X_train  = df.drop(columns='turnover_apos_1_ano')
y_train   = df['turnover_apos_1_ano']

In [ ]:
kf = KFold(n_splits=10, random_state=5, shuffle=True)

# Tuning os parâmetros 



### Floresta aleatorio

In [ ]:
space = [(1,20), # max_depth
        (100,1000),  #n_estimator
        ('auto', 'sqrt', 'log2')] #max_features


In [ ]:
#res1 = gp_minimize(Tuning_RF, space, random_state=40, n_calls=20, verbose=1)




In [ ]:
# Melhor modelo encontrado
model = RandomForestClassifier(n_estimators = 100,max_features = 'sqrt',max_depth=12,random_state = 10)


In [ ]:
acerto1_f1 = np.zeros(10)
acerto1_l  = np.zeros(10)
second_level = np.zeros((X_train.shape[0], 5)) # dados de entrada do modelo 
j = 0
for linhas_treino, linhas_valid in kf.split(X_train):
  acerto1_f1[j] = resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,0.36)
  j =  j+1

print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto1_f1),5),
      ' erro:',np.format_float_positional(np.std(acerto1_f1)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto1_f1),5))  



f1_score 0.81028
f1_score 0.80092
f1_score 0.8139
f1_score 0.79313
f1_score 0.78719
f1_score 0.79812
f1_score 0.79225
f1_score 0.8006
f1_score 0.76341
f1_score 0.78037
dataset f1-score:     0.79402  erro: 0.00441  desvio padrão: 0.01393




* Varrendo o limiar de novo.

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(X_train):
    acerto_c[j] = resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.80207
f1_score 0.79699
f1_score 0.80486
f1_score 0.79234
f1_score 0.78566
f1_score 0.79846
f1_score 0.78867
f1_score 0.79557
f1_score 0.75883
f1_score 0.77325
dataset f1-score:     0.78967  erro: 0.00424  desvio padrão: 0.0134


corte: 0.31
f1_score 0.80089
f1_score 0.79879
f1_score 0.80608
f1_score 0.79199
f1_score 0.78829
f1_score 0.79568
f1_score 0.78867
f1_score 0.79793
f1_score 0.76198
f1_score 0.77414
dataset f1-score:     0.79044  erro: 0.00399  desvio padrão: 0.01262


corte: 0.32
f1_score 0.80178
f1_score 0.79849
f1_score 0.80640
f1_score 0.79382
f1_score 0.78824
f1_score 0.79567
f1_score 0.78927
f1_score 0.79821
f1_score 0.7622
f1_score 0.7734
dataset f1-score:     0.79075  erro: 0.00405  desvio padrão: 0.01281


corte: 0.33
f1_score 0.80328
f1_score 0.79879
f1_score 0.80763
f1_score 0.79102
f1_score 0.78815
f1_score 0.7969
f1_score 0.78862
f1_score 0.79881
f1_score 0.76109
f1_score 0.77558
dataset f1-score:     0.79099  erro: 0.00416  desvio padrão: 0.0

* Removendo a variável deficiência.

In [ ]:
X = X_train.drop(columns = 'func_deficiencia')

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(X):
    acerto_c[j] = resultados_v1(X,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.80119
f1_score 0.79460
f1_score 0.80394
f1_score 0.79661
f1_score 0.78774
f1_score 0.79662
f1_score 0.79077
f1_score 0.79970
f1_score 0.76006
f1_score 0.77011
dataset f1-score:     0.79013  erro: 0.00427  desvio padrão: 0.01349


corte: 0.31
f1_score 0.80149
f1_score 0.79549
f1_score 0.80547
f1_score 0.79722
f1_score 0.78801
f1_score 0.79661
f1_score 0.79077
f1_score 0.80089
f1_score 0.76205
f1_score 0.77036
dataset f1-score:     0.79084  erro: 0.00422  desvio padrão: 0.01334


corte: 0.32
f1_score 0.80179
f1_score 0.79909
f1_score 0.80335
f1_score 0.7966
f1_score 0.79038
f1_score 0.79753
f1_score 0.78984
f1_score 0.80119
f1_score 0.76443
f1_score 0.77351
dataset f1-score:     0.79177  erro: 0.0039  desvio padrão: 0.01233


corte: 0.33
f1_score 0.80269
f1_score 0.80091
f1_score 0.80397
f1_score 0.79535
f1_score 0.78852
f1_score 0.79938
f1_score 0.79167
f1_score 0.80178
f1_score 0.76586
f1_score 0.77695
dataset f1-score:     0.79271  erro: 0.00376  desvio padrão: 0

* O melhor resultado pelo tuning teve os parâmetros n_estimators = 100,max_features = 'sqrt',max_depth=12 . Os resultados obtidos foram:
  * dataset f1-score:     0.79623  erro: 0.00366  desvio padrão: 0.01158
  * dataset log-loss:     0.34227  erro: 0.00392  desvio padrão: 0.01238


### Xgboost

In [ ]:
space = [(1e-3, 1e-1), # lr
          (1, 10),  # max_depth
          (100,1000),  #n_estimator
          (0,1)]    # colsample_bytree

#res2 = gp_minimize(Tuning_XGBR, space, random_state=0, n_calls=50, verbose=1)


In [ ]:
#print('Best Value:',res2.fun)
#print('Parâmetros:',res2.x)

In [ ]:
# Melhor modelo obtido
model = XGBClassifier(colsample_bytree = 1, learning_rate = 0.011484853111591423,
                       max_depth = 5, n_estimators = 268,random_state=40)

In [ ]:
acerto1_f1 = np.zeros(10)
acerto1_l  = np.zeros(10)
second_level = np.zeros((X_train.shape[0], 5)) # dados de entrada do modelo 
j = 0
for linhas_treino, linhas_valid in kf.split(X_train):
  acerto1_f1[j] = resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,0.36)
  j =  j+1

print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto1_f1),5),
      ' erro:',np.format_float_positional(np.std(acerto1_f1)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto1_f1),5))  



f1_score 0.80149
f1_score 0.79367
f1_score 0.80092
f1_score 0.79479
f1_score 0.78738
f1_score 0.79265
f1_score 0.79538
f1_score 0.79672
f1_score 0.76685
f1_score 0.78412
dataset f1-score:     0.7914  erro: 0.00304  desvio padrão: 0.00963


* Varrendo o limiar

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(X_train):
    acerto_c[j] = resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.79474
f1_score 0.79080
f1_score 0.79819
f1_score 0.79242
f1_score 0.78519
f1_score 0.79123
f1_score 0.79084
f1_score 0.79590
f1_score 0.76198
f1_score 0.76923
dataset f1-score:     0.78705  erro: 0.00359  desvio padrão: 0.01134


corte: 0.31
f1_score 0.79589
f1_score 0.79167
f1_score 0.79879
f1_score 0.79211
f1_score 0.78693
f1_score 0.79331
f1_score 0.79205
f1_score 0.79736
f1_score 0.76430
f1_score 0.77262
dataset f1-score:     0.78850  erro: 0.00337  desvio padrão: 0.01067


corte: 0.32
f1_score 0.79647
f1_score 0.79313
f1_score 0.79879
f1_score 0.79331
f1_score 0.78752
f1_score 0.79331
f1_score 0.79387
f1_score 0.79617
f1_score 0.763
f1_score 0.77451
dataset f1-score:     0.78901  erro: 0.00342  desvio padrão: 0.01081


corte: 0.33
f1_score 0.79852
f1_score 0.7952
f1_score 0.79788
f1_score 0.79452
f1_score 0.78987
f1_score 0.79116
f1_score 0.79355
f1_score 0.79468
f1_score 0.76344
f1_score 0.77769
dataset f1-score:     0.78965  erro: 0.00328  desvio padrão: 0.

*  Deletando a variavel deficiência

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(X):
    acerto_c[j] = resultados_v1(X,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.79444
f1_score 0.79021
f1_score 0.79759
f1_score 0.79211
f1_score 0.78725
f1_score 0.78967
f1_score 0.79144
f1_score 0.79590
f1_score 0.76198
f1_score 0.76923
dataset f1-score:     0.78698  erro: 0.00354  desvio padrão: 0.01119


corte: 0.31
f1_score 0.79472
f1_score 0.79167
f1_score 0.80060
f1_score 0.79271
f1_score 0.78693
f1_score 0.79148
f1_score 0.79205
f1_score 0.79472
f1_score 0.76256
f1_score 0.77262
dataset f1-score:     0.78801  erro: 0.00346  desvio padrão: 0.01095


corte: 0.32
f1_score 0.79647
f1_score 0.79254
f1_score 0.79788
f1_score 0.79452
f1_score 0.78693
f1_score 0.79148
f1_score 0.79387
f1_score 0.79589
f1_score 0.76453
f1_score 0.77451
dataset f1-score:     0.78886  erro: 0.00327  desvio padrão: 0.01033


corte: 0.33
f1_score 0.79794
f1_score 0.7952
f1_score 0.79697
f1_score 0.79421
f1_score 0.78869
f1_score 0.78931
f1_score 0.79355
f1_score 0.79529
f1_score 0.76579
f1_score 0.77769
dataset f1-score:     0.78946  erro: 0.00306  desvio padrão: 

###MLP

In [ ]:
#space = [('identity', 'logistic', 'tanh', 'relu'), 
#         (1,500),
#         (0,1e-3)]

#res3 = gp_minimize(Tuning_MLP, space, random_state=0, n_calls=30, verbose=1)

In [ ]:
#print('Best Value:',res3.fun)
#print('Parâmetros:',res3.x)

In [ ]:


#clf   = MLPClassifier(hidden_layer_sizes =[1],activation='tanh',solver='lbfgs',alpha=0,max_iter=500,random_state=40)
#model = make_pipeline(MinMaxScaler(), clf)

clf   = MLPClassifier(hidden_layer_sizes =[261],activation='relu',solver='adam',alpha= 0.0006788795301189604,max_iter=500)
model = make_pipeline(MinMaxScaler(), clf)

In [ ]:
acerto1_f1 = np.zeros(10)
acerto1_l  = np.zeros(10)
second_level = np.zeros((X_train.shape[0], 5)) # dados de entrada do modelo 
j = 0
for linhas_treino, linhas_valid in kf.split(X_train):
  acerto1_f1[j] = resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,0.36)
  j =  j+1

print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto1_f1),5),
      ' erro:',np.format_float_positional(np.std(acerto1_f1)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto1_f1),5))  

f1_score 0.79846
f1_score 0.79274
f1_score 0.79939
f1_score 0.78951
f1_score 0.77596
f1_score 0.77924
f1_score 0.78456
f1_score 0.79024
f1_score 0.75534
f1_score 0.73901
dataset f1-score:     0.78045  erro: 0.00583  desvio padrão: 0.01843


* Varrendo o limiar.

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(X_train):
    acerto_c[j] = resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.79356
f1_score 0.79412
f1_score 0.80956
f1_score 0.78886
f1_score 0.7836
f1_score 0.78502
f1_score 0.78407
f1_score 0.79056
f1_score 0.75272
f1_score 0.74881
dataset f1-score:     0.78309  erro: 0.00559  desvio padrão: 0.01767


corte: 0.31
f1_score 0.80091
f1_score 0.79034
f1_score 0.80677
f1_score 0.78366
f1_score 0.79109
f1_score 0.78481
f1_score 0.78045
f1_score 0.78593
f1_score 0.74441
f1_score 0.7601
dataset f1-score:     0.78285  erro: 0.00551  desvio padrão: 0.01743


corte: 0.32
f1_score 0.79912
f1_score 0.79031
f1_score 0.80897
f1_score 0.78746
f1_score 0.77447
f1_score 0.78626
f1_score 0.79106
f1_score 0.79262
f1_score 0.74346
f1_score 0.75851
dataset f1-score:     0.78322  erro: 0.00584  desvio padrão: 0.01847


corte: 0.33
f1_score 0.79617
f1_score 0.78280
f1_score 0.81015
f1_score 0.78631
f1_score 0.78144
f1_score 0.78095
f1_score 0.78365
f1_score 0.79204
f1_score 0.75267
f1_score 0.75125
dataset f1-score:     0.78174  erro: 0.00541  desvio padrão: 0

* Excluindo a variável deficiência.

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(X):
    acerto_c[j] = resultados_v1(X,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.8018
f1_score 0.78669
f1_score 0.80519
f1_score 0.78419
f1_score 0.78024
f1_score 0.78626
f1_score 0.78366
f1_score 0.79121
f1_score 0.73824
f1_score 0.74836
dataset f1-score:     0.78058  erro: 0.0064  desvio padrão: 0.02023


corte: 0.31
f1_score 0.79351
f1_score 0.79353
f1_score 0.80727
f1_score 0.78806
f1_score 0.78146
f1_score 0.78179
f1_score 0.78659
f1_score 0.79212
f1_score 0.75739
f1_score 0.75119
dataset f1-score:     0.78329  erro: 0.00510  desvio padrão: 0.01613


corte: 0.32
f1_score 0.8065
f1_score 0.78577
f1_score 0.80613
f1_score 0.78959
f1_score 0.78719
f1_score 0.78825
f1_score 0.78115
f1_score 0.78510
f1_score 0.74921
f1_score 0.75423
dataset f1-score:     0.78331  erro: 0.00562  desvio padrão: 0.01778


corte: 0.33
f1_score 0.79130
f1_score 0.78528
f1_score 0.78842
f1_score 0.78631
f1_score 0.78843
f1_score 0.78517
f1_score 0.78686
f1_score 0.7852
f1_score 0.75057
f1_score 0.75348
dataset f1-score:     0.78010  erro: 0.00448  desvio padrão: 0.0

* Deletando a variavel deficiêntes

* Pelo resultados obtidos da rede neural foi visto que o melhor desepenho foi alcançando quando existe punição de variáveis. O XGB tem como fazer isso também, por esse motivo será visto se ao criar essa penalização o seu desepenho melhorada.

### XGB punindo variáveis

In [ ]:
#space = [(1e-3, 1e-1), # lr
#          (1, 10),     # max_depth
#          (100,1000),  # n_estimator
#          (0,1),       # colsample_bytree
#          (0,1e-3),    # reg_lambda
#          (0,1e-3)]    # reg_alpha


In [ ]:
#res4 = gp_minimize(Tuning_XGBR1, space, random_state=0, n_calls=30, verbose=1)

In [ ]:
#print('Best Value:',res4.fun)
#print('Parâmetros:',res4.x)

In [ ]:
lr = 0.027840476080879074
max_depth = 6
n_estimators = 669
colsample_bytree = 1
lamb             = 0.0009938152575176688
alpha            = 0.0


In [ ]:
model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)

In [ ]:
acerto1_f1 = np.zeros(10)
acerto1_l  = np.zeros(10)
second_level = np.zeros((X_train.shape[0], 5)) # dados de entrada do modelo 
j = 0
for linhas_treino, linhas_valid in kf.split(X_train):
  acerto1_f1[j] = resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,0.36)
  j =  j+1

print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto1_f1),5),
      ' erro:',np.format_float_positional(np.std(acerto1_f1)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto1_f1),5))  

f1_score 0.8071
f1_score 0.79969
f1_score 0.81665
f1_score 0.80128
f1_score 0.79373
f1_score 0.7984
f1_score 0.79554
f1_score 0.81712
f1_score 0.77425
f1_score 0.78455
dataset f1-score:     0.79883  erro: 0.00397  desvio padrão: 0.01254


* Varrendo o limiar

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(X_train):
    acerto_c[j] = resultados_v1(X_train,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.80423
f1_score 0.79509
f1_score 0.81128
f1_score 0.79688
f1_score 0.79321
f1_score 0.80000
f1_score 0.79595
f1_score 0.81442
f1_score 0.76997
f1_score 0.78517
dataset f1-score:     0.79662  erro: 0.00382  desvio padrão: 0.01207


corte: 0.31
f1_score 0.80423
f1_score 0.79754
f1_score 0.81452
f1_score 0.79781
f1_score 0.79382
f1_score 0.79716
f1_score 0.79499
f1_score 0.81567
f1_score 0.7712
f1_score 0.7865
dataset f1-score:     0.79734  erro: 0.00387  desvio padrão: 0.01224


corte: 0.32
f1_score 0.80455
f1_score 0.79567
f1_score 0.81581
f1_score 0.79905
f1_score 0.79535
f1_score 0.79430
f1_score 0.79624
f1_score 0.81692
f1_score 0.77233
f1_score 0.78372
dataset f1-score:     0.79739  erro: 0.00401  desvio padrão: 0.01268


corte: 0.33
f1_score 0.8067
f1_score 0.79658
f1_score 0.81775
f1_score 0.79779
f1_score 0.79627
f1_score 0.79302
f1_score 0.7978
f1_score 0.81481
f1_score 0.77346
f1_score 0.78401
dataset f1-score:     0.79782  erro: 0.00397  desvio padrão: 0.0

Excluido a variável deficiênte

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(X):
    acerto_c[j] = resultados_v1(X,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.80545
f1_score 0.79693
f1_score 0.81279
f1_score 0.79497
f1_score 0.79540
f1_score 0.79653
f1_score 0.79405
f1_score 0.81283
f1_score 0.77424
f1_score 0.78458
dataset f1-score:     0.79678  erro: 0.00355  desvio padrão: 0.01122


corte: 0.31
f1_score 0.80638
f1_score 0.79722
f1_score 0.81441
f1_score 0.79590
f1_score 0.79446
f1_score 0.79747
f1_score 0.79592
f1_score 0.81442
f1_score 0.77450
f1_score 0.78421
dataset f1-score:     0.79749  erro: 0.00370  desvio padrão: 0.01171


corte: 0.32
f1_score 0.80822
f1_score 0.79598
f1_score 0.8189
f1_score 0.79335
f1_score 0.79537
f1_score 0.80000
f1_score 0.79497
f1_score 0.81476
f1_score 0.77626
f1_score 0.78686
dataset f1-score:     0.79847  erro: 0.00382  desvio padrão: 0.01209


corte: 0.33
f1_score 0.80883
f1_score 0.8
f1_score 0.81804
f1_score 0.79650
f1_score 0.79505
f1_score 0.79904
f1_score 0.79213
f1_score 0.81510
f1_score 0.77617
f1_score 0.78686
dataset f1-score:     0.79877  erro: 0.00382  desvio padrão: 0.01

### Análisando os valores:
---
* Fl aleatoria 1: 0.79623  erro: 0.00366  desvio padrão: 0.01158 (Limiar 0.42)
* Fl aleatoria 2: 0.79461  erro: 0.00418  desvio padrão: 0.01322 (Limiar 0.42)
---
* Xgboost 1: 0.79521  erro: 0.00324  desvio padrão: 0.01024 (Limiar 0.41)
* Xgboost 2: 0.79518  erro: 0.00319  desvio padrão: 0.01010 (Limiar 0.41)
---
* MLP 1: 0.78502  erro: 0.00455  desvio padrão: 0.01439 (Limiar 0.41)
* MLP 2:  0.78566  erro: 0.00535  desvio padrão: 0.01692 (Limiar 0.38)
---
* Xgboost v2 1:  0.79986  erro: 0.00396  desvio padrão: 0.01253  (Limiar 0.38)
* Xgboost v2 2 : 0.79964  erro: 0.00422  desvio padrão: 0.01335 (Limiar 0.35)
---
* O melhor modelo obtido foi o Xgboost V2 1. E um ponto que se destaca nesses resutados é que o não uso da variável deficiência causa um impacto negativo muito baixo desepenho de cada modelo.  


# Stacking

### Modelos utilizados

In [ ]:
model1 = RandomForestClassifier(n_estimators = 100,max_features = 'sqrt',max_depth=12,random_state = 10)
model2 = XGBClassifier(colsample_bytree = 1, learning_rate = 0.011484853111591423,
                       max_depth = 5, n_estimators = 268,random_state=40)

clf    = MLPClassifier(hidden_layer_sizes =[261],activation='relu',solver='adam',alpha= 0.0006788795301189604,max_iter=500,random_state=40)
model3 = make_pipeline(MinMaxScaler(), clf)

clf    = MLPClassifier(hidden_layer_sizes =[1],activation='tanh',solver='lbfgs',alpha=0,max_iter=500,random_state=40)
model4 = make_pipeline(MinMaxScaler(), clf)

lr = 0.027840476080879074
max_depth = 6
n_estimators = 669
colsample_bytree = 1
lamb             = 0.0009938152575176688
alpha            = 0.0

model5 = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)








acertof1 = np.zeros(10)
acertol  = np.zeros(10)
second_level = np.zeros((X_train.shape[0], 5)) # dados de entrada do modelo 
j = 0
for linhas_treino, linhas_valid in kf.split(X_train):
  prf1 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model1)
  prf2 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model2)
  prf3 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model3)
  prf4 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model4)
  prf5 = resultados_v2(X_train,y_train,linhas_treino,linhas_valid,model5)
  
  j =  j+1

  second_level[linhas_valid, 0] = prf1
  second_level[linhas_valid, 1] = prf2
  second_level[linhas_valid, 2] = prf3
  second_level[linhas_valid, 3] = prf4
  second_level[linhas_valid, 4] = prf5
 

### Segundo nível: Seleção de feacuter

In [ ]:
df_staking = pd.DataFrame(second_level,columns=['Fl', 'XGBM1', 'MLP1', 'MLP2','XGBM2'])

In [ ]:
df_staking.corr()

,Fl,XGBM1,MLP1,MLP2,XGBM2
Fl,1.000000,0.984669,0.958504,0.911042,0.973758
XGBM1,0.984669,1.000000,0.962511,0.930420,0.967946
MLP1,0.958504,0.962511,1.000000,0.929560,0.940234
MLP2,0.911042,0.930420,0.929560,1.000000,0.877770
XGBM2,0.973758,0.967946,0.940234,0.877770,1.000000


* As correlações estão muito alta, talvez o desepenho de um staking com esses modelos não vai diferenciar muito dos demais modelos já utilizados. Mas mesmo assim será visto como ele vai se empenhar já que ele traz resultados mais estaveis.

In [ ]:
model = LogisticRegression()

* Antes de começar a seleção de variáveis será visto qual o melhor ponto de corte.

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(df_staking):
    acerto_c[j] = resultados_v1(df_staking,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.81419
f1_score 0.79751
f1_score 0.81517
f1_score 0.80253
f1_score 0.79128
f1_score 0.79493
f1_score 0.79842
f1_score 0.80463
f1_score 0.76810
f1_score 0.7884
dataset f1-score:     0.79752  erro: 0.00407  desvio padrão: 0.01288


corte: 0.31
f1_score 0.81424
f1_score 0.79875
f1_score 0.81681
f1_score 0.80317
f1_score 0.79313
f1_score 0.79745
f1_score 0.79715
f1_score 0.80403
f1_score 0.76873
f1_score 0.78767
dataset f1-score:     0.79811  erro: 0.00409  desvio padrão: 0.01292


corte: 0.32
f1_score 0.81366
f1_score 0.79938
f1_score 0.81558
f1_score 0.80128
f1_score 0.79154
f1_score 0.7952
f1_score 0.79683
f1_score 0.80653
f1_score 0.76898
f1_score 0.78767
dataset f1-score:     0.79766  erro: 0.00405  desvio padrão: 0.01280


corte: 0.33
f1_score 0.81684
f1_score 0.79969
f1_score 0.81695
f1_score 0.80160
f1_score 0.79121
f1_score 0.79582
f1_score 0.79808
f1_score 0.80281
f1_score 0.76822
f1_score 0.7859
dataset f1-score:     0.79771  erro: 0.00428  desvio padrão: 0.

In [ ]:
for i in range(df_staking.shape[1]):
  acerto = np.zeros(10)
  j = 0
  print(df_staking.columns[i])
  for linhas_treino, linhas_valid in kf.split(df_staking):
    acerto[j] =  resultados_v1(pd.DataFrame(df_staking[df_staking.columns[i]]),y_train,linhas_treino,linhas_valid,model,0.31)
    j =  j+1
  
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

Fl
f1_score 0.80805
f1_score 0.79938
f1_score 0.81265
f1_score 0.80032
f1_score 0.78722
f1_score 0.79936
f1_score 0.79681
f1_score 0.80216
f1_score 0.7731
f1_score 0.78179
dataset est_1 f1-score:     0.79608  erro: 0.00361  desvio padrão: 0.01143


XGBM1
f1_score 0.80913
f1_score 0.79969
f1_score 0.80904
f1_score 0.79717
f1_score 0.78362
f1_score 0.79937
f1_score 0.79688
f1_score 0.79908
f1_score 0.77240
f1_score 0.78445
dataset est_1 f1-score:     0.79508  erro: 0.00348  desvio padrão: 0.01102


MLP1
f1_score 0.80217
f1_score 0.78817
f1_score 0.80806
f1_score 0.79324
f1_score 0.78187
f1_score 0.7896
f1_score 0.78394
f1_score 0.7845
f1_score 0.73379
f1_score 0.74660
dataset est_1 f1-score:     0.78119  erro: 0.00699  desvio padrão: 0.02211


MLP2
f1_score 0.78935
f1_score 0.78528
f1_score 0.80406
f1_score 0.78893
f1_score 0.78301
f1_score 0.77857
f1_score 0.77107
f1_score 0.77123
f1_score 0.75079
f1_score 0.75314
dataset est_1 f1-score:     0.77754  erro: 0.00496  desvio padrão: 0.0156

* Melhor modelo XGBM2

In [ ]:
X = df_staking.copy()

In [ ]:
col = ['Fl', 'MLP1', 'MLP2', 'XGBM1']

In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X):
    
    acerto[j] = resultados_v1(X[['XGBM2',col[i]]],y_train,linhas_treino,linhas_valid,model,0.31)
    j =  j+1

  
  
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

Fl
f1_score 0.8147
f1_score 0.79937
f1_score 0.81891
f1_score 0.80321
f1_score 0.78712
f1_score 0.80065
f1_score 0.7968
f1_score 0.80871
f1_score 0.77351
f1_score 0.78095
dataset est_1 f1-score:     0.79839  erro: 0.00433  desvio padrão: 0.0137


MLP1
f1_score 0.80881
f1_score 0.80383
f1_score 0.81494
f1_score 0.80032
f1_score 0.79213
f1_score 0.79773
f1_score 0.79391
f1_score 0.80693
f1_score 0.76793
f1_score 0.78194
dataset est_1 f1-score:     0.79685  erro: 0.00415  desvio padrão: 0.01313


MLP2
f1_score 0.81148
f1_score 0.80032
f1_score 0.81505
f1_score 0.7952
f1_score 0.79022
f1_score 0.79518
f1_score 0.79424
f1_score 0.80811
f1_score 0.77250
f1_score 0.7859
dataset est_1 f1-score:     0.79682  erro: 0.00381  desvio padrão: 0.01205


XGBM1
f1_score 0.81356
f1_score 0.79813
f1_score 0.81423
f1_score 0.80221
f1_score 0.79407
f1_score 0.79650
f1_score 0.79779
f1_score 0.80989
f1_score 0.7706
f1_score 0.78943
dataset est_1 f1-score:     0.79864  erro: 0.00388  desvio padrão: 0.01226



* O XGBM2  esta obtendo o melhor resultado.

In [ ]:
col = ['Fl','MLP1', 'MLP2']

In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X):
    
    acerto[j] = resultados_v1(X[['XGBM1','XGBM2',col[i]]],y_train,linhas_treino,linhas_valid,model,0.31)
    j =  j+1

  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

Fl
f1_score 0.81458
f1_score 0.79938
f1_score 0.81334
f1_score 0.80285
f1_score 0.79095
f1_score 0.7968
f1_score 0.79683
f1_score 0.80868
f1_score 0.77110
f1_score 0.78485
dataset est_1 f1-score:     0.79794  erro: 0.004  desvio padrão: 0.01264


MLP1
f1_score 0.81516
f1_score 0.79969
f1_score 0.81452
f1_score 0.80222
f1_score 0.79469
f1_score 0.79714
f1_score 0.79874
f1_score 0.80769
f1_score 0.76672
f1_score 0.78912
dataset est_1 f1-score:     0.79857  erro: 0.00418  desvio padrão: 0.01322


MLP2
f1_score 0.81356
f1_score 0.79969
f1_score 0.81452
f1_score 0.79779
f1_score 0.79283
f1_score 0.79937
f1_score 0.79684
f1_score 0.80890
f1_score 0.76973
f1_score 0.78876
dataset est_1 f1-score:     0.7982  erro: 0.00394  desvio padrão: 0.01245




* Os resultados mostram que a MLP1, MLP2 e a Rl não estão causando nenhum impacto na previsão. Agora será testato o XGBM e a MLP como segundo nível.

* Resultado:
  * Tadas as variáveiis: 0.79800  erro: 0.00426  desvio padrão: 0.01346
  * Selecionaodo as variáveis: 0.79833  erro: 0.0039  desvio padrão: 0.01232

* Agora será visto outros modelos modelos como segundo nível.


### Segundo nível: MLP

In [ ]:
X_train = df_staking

In [ ]:
#space = [('identity', 'logistic', 'tanh', 'relu'), 
#         (1,500),
#         (0,1e-3)]

#res4 = gp_minimize(Tuning_MLP, space, random_state=0, n_calls=30, verbose=1)

In [ ]:
#print('Best Value:',res4.fun)
#print('Parâmetros:',res4.x)

In [ ]:
clf   = MLPClassifier(hidden_layer_sizes =[54],activation='relu',solver='adam',alpha=  0.0004736004193466575,max_iter=500)
model = make_pipeline(MinMaxScaler(), clf)

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(df_staking):
    acerto_c[j] = resultados_v1(df_staking,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.80514
f1_score 0.79908
f1_score 0.81240
f1_score 0.7938
f1_score 0.7966
f1_score 0.80221
f1_score 0.79876
f1_score 0.80633
f1_score 0.76653
f1_score 0.78109
dataset f1-score:     0.79619  erro: 0.00401  desvio padrão: 0.01269


corte: 0.31
f1_score 0.80760
f1_score 0.79846
f1_score 0.81308
f1_score 0.79441
f1_score 0.79908
f1_score 0.80187
f1_score 0.79619
f1_score 0.80822
f1_score 0.77048
f1_score 0.78045
dataset f1-score:     0.79699  erro: 0.00389  desvio padrão: 0.0123


corte: 0.32
f1_score 0.80890
f1_score 0.80217
f1_score 0.81245
f1_score 0.79689
f1_score 0.79753
f1_score 0.80251
f1_score 0.7975
f1_score 0.81044
f1_score 0.76936
f1_score 0.78356
dataset f1-score:     0.79813  erro: 0.00394  desvio padrão: 0.01245


corte: 0.33
f1_score 0.80579
f1_score 0.80093
f1_score 0.81383
f1_score 0.79529
f1_score 0.79753
f1_score 0.80095
f1_score 0.79811
f1_score 0.8067
f1_score 0.76935
f1_score 0.78195
dataset f1-score:     0.79704  erro: 0.00385  desvio padrão: 0.01

**Removendo as variáveis Rl,MLP1,MLP2**

In [ ]:
X = df_staking[['XGBM1','XGBM2']]

In [ ]:
j = 0
for linhas_treino, linhas_valid in kf.split(df_staking):
    acerto_c[j] = resultados_v1(X,y_train,linhas_treino,linhas_valid,model,0.35)
    j =  j+1

print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
print('\n')

f1_score 0.80890
f1_score 0.79816
f1_score 0.81688
f1_score 0.79968
f1_score 0.78869
f1_score 0.80223
f1_score 0.79968
f1_score 0.81081
f1_score 0.77461
f1_score 0.78644
dataset f1-score:     0.79861  erro: 0.00378  desvio padrão: 0.01194




### Segundo nível : Xgboost

In [ ]:
#space = [(1e-3, 1e-1), # lr
#          (1, 10),     # max_depth
#          (100,1000),  # n_estimator
#          (0,1),       # colsample_bytree
#          (0,1e-3),    # reg_lambda
#          (0,1e-3)]    # reg_alpha


In [ ]:
#res5 = gp_minimize(Tuning_XGBR1, space, random_state=0, n_calls=60, verbose=1)

In [ ]:
#print('Best Value:',res5.fun)
#print('Parâmetros:',res5.x)

In [ ]:
lr = 0.04248763694738395
max_depth = 1
n_estimators = 1000
colsample_bytree = 1
reg_lambda = 4.222462269882505e-05
reg_alpha = 0.1

model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)

* variando o limiar

In [ ]:
for i in np.arange(0.3,0.51,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)

  for linhas_treino, linhas_valid in kf.split(df_staking):
    acerto_c[j] = resultados_v1(df_staking,y_train,linhas_treino,linhas_valid,model,i)
    j =  j+1

  print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
  print('\n')

corte: 0.3
f1_score 0.80691
f1_score 0.79667
f1_score 0.80982
f1_score 0.79414
f1_score 0.79268
f1_score 0.80062
f1_score 0.79507
f1_score 0.80451
f1_score 0.76633
f1_score 0.78017
dataset f1-score:     0.79469  erro: 0.00392  desvio padrão: 0.01238


corte: 0.31
f1_score 0.80813
f1_score 0.7997
f1_score 0.81077
f1_score 0.79475
f1_score 0.79084
f1_score 0.80186
f1_score 0.79598
f1_score 0.80422
f1_score 0.76753
f1_score 0.78146
dataset f1-score:     0.79552  erro: 0.00391  desvio padrão: 0.01236


corte: 0.32
f1_score 0.81057
f1_score 0.80061
f1_score 0.81019
f1_score 0.79473
f1_score 0.79052
f1_score 0.80156
f1_score 0.79628
f1_score 0.80514
f1_score 0.76875
f1_score 0.78405
dataset f1-score:     0.79624  erro: 0.00383  desvio padrão: 0.01211


corte: 0.33
f1_score 0.81057
f1_score 0.80305
f1_score 0.81206
f1_score 0.79439
f1_score 0.7902
f1_score 0.80189
f1_score 0.7969
f1_score 0.80484
f1_score 0.76923
f1_score 0.7853
dataset f1-score:     0.79684  erro: 0.00387  desvio padrão: 0.0

In [ ]:
j = 0
for linhas_treino, linhas_valid in kf.split(df_staking):
    acerto_c[j] = resultados_v1(X,y_train,linhas_treino,linhas_valid,model,0.4)
    j =  j+1

print('dataset f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
      ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))  
print('\n')

f1_score 0.81206
f1_score 0.79937
f1_score 0.81701
f1_score 0.79776
f1_score 0.79278
f1_score 0.79936
f1_score 0.79712
f1_score 0.81401
f1_score 0.77531
f1_score 0.7872
dataset f1-score:     0.7992  erro: 0.00383  desvio padrão: 0.01210




### Avaliando os resultaods:

* Stacking  lr: 0.79833  erro: 0.0039  desvio padrão: 0.01232 (Corte 0.31)
* Stacking MLP: 0.79861  erro: 0.00378  desvio padrão: 0.01194  (Corte 0.35)
* Stacking XGBoost : 0.7992  erro: 0.00383  desvio padrão: 0.01210 (Corte 0.40)


# Conclusão:

* Os melhores despeenhos foram obtidos pelo um Xboots que obteve o desepeho de **0.79964 erro: 0.00422 desvio padrão: 0.01335 com um limiar de 0.35** o outro resultado foi o um Stacking que utilizou os dois modelos encontrados de Xboost como primeiro nível e o segundo nível foi o XGBoost que teve como o resultado um valor de f1 de **0.7992 erro: 0.00383 desvio padrão: 0.01210**, com um limiar de 0.4.

* Para ter uma ideia como esta sendo o desepenho desses modelos nos dados de teste da competição, será mandado o XGBoost.




# Competição

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
def encoder_var(df,flag):
  if flag == 1:
    ind = df.columns[df.dtypes == 'object']
  if flag == 0:
     ind = df.columns[df.dtypes == 'category']

  for i in range(len(ind)):
    enc = LabelEncoder()
    inteiros = enc.fit_transform(df[ind[i]])
    df[ind[i]] = inteiros

  return df

def resultados(X_train,y_train,linhas_treino,linhas_valid,flag,coluna,suport,model,C):

  if suport == 'Dataframe':
    X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
    y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

  if suport == 'array':
    X_t, X_v = X_train[linhas_treino], X_train[linhas_valid]
    y_t, y_v = y_train[linhas_treino], y_train[linhas_valid]

  if flag == 'CountEncoder':
      cenc = CountEncoder(normalize=False)
      X_t[coluna] = cenc.fit_transform(X_t[coluna],y_t)
      X_v[coluna] = cenc.transform(X_v[coluna])

  if flag == 'TargetEncoder':
      tenc = TargetEncoder()
      X_t[coluna] = tenc.fit_transform(X_t[coluna],y_t)
      X_v[coluna] = tenc.transform(X_v[coluna])


  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > C).astype(int)
  acerto_v = f1_score(y_v, prf_)
  return acerto_v

def tratamento(df, flag):

  df['func_idade_log'] = np.log(df['func_idade'])
  df['contrato_horastrabalho_log'] = np.log(df['contrato_horastrabalho'] + 1)
  df['contrato_salario_log'] = np.log10(df['contrato_salario'])
  if flag == 1:
    df['turnover_apos_1_ano'] = df['turnover_apos_1_ano'].map({'NAO': 0, 'SIM': 1})
  
  df['func_deficiencia'] = df['func_deficiencia'].map({'NAO': 0, 'SIM': 1})
  df['func_sexo'] = df['func_sexo'].map({'FEMININO': 0, 'MASCULINO': 1})

  df = df.drop(columns = ['contrato_horastrabalho_log'])

  df = encoder_var(df,1)

  col2 = ['func_idade' ,'contrato_salario_log']
  df = df.drop(columns = col2)

  df['contrato_horastrabalho'].loc[df['contrato_horastrabalho'] == 0] =df['contrato_horastrabalho'].loc[df['contrato_horastrabalho'] > 0].mean()

  df['contrato_horastrabalho']= pd.cut(df["contrato_horastrabalho"], bins= 12)

  print(np.unique(df['contrato_horastrabalho']))

  df = encoder_var(df,0)

  return df

In [ ]:
df = pd.read_csv('/content/drive/My Drive/competição flai nova/treino.csv',encoding = "UTF-8", sep = ";",decimal = ',',dayfirst = True)

In [ ]:
df_teste = pd.read_csv('/content/drive/My Drive/competição flai nova/teste.csv',encoding = "UTF-8", sep = ";",decimal = ',',dayfirst = True)

In [ ]:
df = tratamento(df,1)

[Interval(0.957, 4.583, closed='right')
 Interval(4.583, 8.167, closed='right')
 Interval(8.167, 11.75, closed='right')
 Interval(11.75, 15.333, closed='right')
 Interval(15.333, 18.917, closed='right')
 Interval(18.917, 22.5, closed='right')
 Interval(22.5, 26.083, closed='right')
 Interval(26.083, 29.667, closed='right')
 Interval(29.667, 33.25, closed='right')
 Interval(33.25, 36.833, closed='right')
 Interval(36.833, 40.417, closed='right')
 Interval(40.417, 44.0, closed='right')]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df_teste = tratamento(df_teste,0)

[Interval(0.957, 4.583, closed='right')
 Interval(4.583, 8.167, closed='right')
 Interval(8.167, 11.75, closed='right')
 Interval(11.75, 15.333, closed='right')
 Interval(15.333, 18.917, closed='right')
 Interval(18.917, 22.5, closed='right')
 Interval(22.5, 26.083, closed='right')
 Interval(26.083, 29.667, closed='right')
 Interval(29.667, 33.25, closed='right')
 Interval(33.25, 36.833, closed='right')
 Interval(36.833, 40.417, closed='right')
 Interval(40.417, 44.0, closed='right')]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
X_train  = df.drop(columns='turnover_apos_1_ano')
y_train = df['turnover_apos_1_ano']

In [ ]:
lr = 0.027840476080879074
max_depth = 6
n_estimators = 669
colsample_bytree = 1
lamb             = 0.0009938152575176688
alpha            = 0.0
model = XGBClassifier(colsample_bytree = colsample_bytree, learning_rate = lr,
                       max_depth = max_depth, n_estimators = n_estimators,reg_lambda = lamb, reg_alpha = alpha,random_state=40)

In [ ]:
model = model.fit(X_train, y_train)
prf = model.predict_proba(df_teste)[:,1]
prf_ = (prf > 0.38).astype(int)


In [ ]:
subm = pd.DataFrame()
subm['turnover_apos_1_ano'] = prf_

In [ ]:
subm['turnover_apos_1_ano'] = subm['turnover_apos_1_ano'].map({0:'NAO', 1: 'SIM'})

In [ ]:
subm

,turnover_apos_1_ano
0,SIM
1,NAO
2,NAO
3,NAO
4,NAO
...,...
14995,NAO
14996,SIM
14997,SIM
14998,NAO


In [ ]:
subm.to_csv('/content/drive/My Drive/competição flai nova/submissão.csv', index = False)

* Esse modelo teve um desepeho de 0.798 na competição.